<div align="center">

# Set Partitioning
<h4>
  Wesley Dyk<br>
  <small style="font-weight: normal;">
    Senior Quantum Solutions Architect<br>
    Quantum Computing Inc.
  </small>
</h4>

<br>

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/qci-wdyk/eqc-models-tutorial/blob/main/tutorial05-setpartition.ipynb)

</div>


## Imports

In [1]:
try:
    import eqc_models
except ImportError:
    !pip install eqc-models
import os
import numpy as np
from eqc_models.combinatorics import SetPartitionModel
from eqc_models.solvers import Dirac3IntegerCloudSolver
try:
    from google.colab import userdata
except ImportError:
    userdata = None

## API Keys

In [2]:
# Define the API URL and token  for QCI
api_url ="https://api.qci-prod.com"
if userdata is None:
    api_token = "" # replace or use environment variables to configure
else:
    api_token = userdata.get("QCI_TOKEN")
    os.environ["QCI_TOKEN"] = api_token
    os.environ["QCI_API_URL"] = api_url

The `SetPartitionModel` is a tool for solving set partitioning optimization problems.

Given a collection of **subsets** and their corresponding **weights**, this class finds the lowest-cost combination of subsets that perfectly covers every element in a larger universal set. 
It automatically formulates the problem's constraints and objective function, preparing it for use with an optimization solver.

In [3]:
S = [{"A", "B"}, {"A", "C"}, {"B"}, {"C"}]
weights = [4, 4, 1, 3]

The constraints of the partition model ensure that every element in the universal set is covered exactly once.
The `penalty_multiplier` sets the "punishment" for breaking the constraint.

In [4]:
model = SetPartitionModel(S, weights)
model.penalty_multiplier = 10

In [5]:
solver = Dirac3IntegerCloudSolver(url=api_url, api_token=api_token)

In [6]:
response = solver.solve(model, relaxation_schedule=1, num_samples=5)

2025-09-03 12:19:21 - Dirac allocation balance = 0 s (unmetered)
2025-09-03 12:19:21 - Job submitted: job_id='68b886a98060c93397963825'
2025-09-03 12:19:21 - QUEUED
2025-09-03 12:19:24 - RUNNING
2025-09-03 12:19:26 - COMPLETED
2025-09-03 12:19:29 - Dirac allocation balance = 0 s (unmetered)


The output of the `SetPartitionModel` selects a combination of the subsets that can cover most elements in the universal set, while minimizing the total weight of the selected subsets.

In [7]:
solution = response["results"]["solutions"][0]
U = set()

for i in range(len(solution)):
    if solution[i] == 1:
        U = U.union(S[i])
solution, U

([0, 1, 1, 0], {'A', 'B', 'C'})

In [8]:
[S[i] for i in range(len(S)) if solution[i] == 1]

[{'A', 'C'}, {'B'}]